# Loading a sample of data into Argilla 

Loads some unlabelled text blocks into Argilla for an entity annotation task. For demo purposes.

In [ ]:
# installing argilla in the notebook kernel as there is a dependency clash when trying to install with poetry
import sys

!{sys.executable} -m pip install argilla

In [1]:
import os

from cpr_data_access.models import Dataset
from dotenv import load_dotenv, find_dotenv
import argilla as rg
from tqdm.auto import tqdm

load_dotenv(find_dotenv())

True

In [3]:
# User management is done at a workspace level
rg.init(workspace="gst", api_key=os.environ["ARGILLA_API_KEY"])
dataset = Dataset().load_from_local(os.environ["CPR_DATA_DIR"], limit=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
records = []

for document in tqdm(dataset.documents):
    if document.text_blocks is None:
        print(f"Skipping {document.document_id} as no text blocks")
        continue

    doc_metadata = document.dict(exclude={"text_blocks", "page_metadata"})

    # FIXME: use spacy for tokenization
    for block in document.text_blocks:
        block_metadata = block.dict(exclude={"text"})
        block_text = block.to_string()

        records.append(
            rg.TokenClassificationRecord(
                text=block_text,
                tokens=block_text.split(" "),
                metadata=doc_metadata | block_metadata,
                id=f"{block.text_block_id}_{document.document_id}",
            )
        )

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/kalyan/.pyenv/versions/3.9.10/lib/python3.9/site-packages/argilla/client/models.py:74: UserWarning: Some metadata values could exceed the max length. For those cases, values will be truncated by keeping only the last 128 characters. You can configure this length in the server with the ARGILLA_METADATA_FIELD_LENGTH environment variable. Note that, setting this too high may lead to Elastic performance issues.
  warnings.warn(message, UserWarning)


Skipping CCLW.executive.9783.4434 as no text blocks


In [6]:
dataset_metadata = {
    "documents": [doc.document_id for doc in dataset.documents],
}

rg.log(
    records,
    name="demo_unlabelled",
    metadata=dataset_metadata,
)

  0%|          | 0/2760 [00:00<?, ?it/s]

2760 records logged to http://localhost:6900/datasets/gst/demo_unlabelled


BulkResponse(dataset='demo_unlabelled', processed=2760, failed=0)